In [1]:
import gc, cv2
import numpy as np
import pandas as pd
import tensorflow as tf

from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, BatchNormalization, MaxPooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array , array_to_img
from PIL import Image
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import os , zipfile
os.getcwd()

'/home/roshanbtech/Deep_Learning/Frenship_detection'

In [177]:
with zipfile.ZipFile('/home/roshanbtech/Deep_Learning/Frenship_detection/806312_1381964_bundle_archive.zip', 'r') as zip_ref:
    zip_ref.extractall('/home/roshanbtech/Deep_Learning/Frenship_detection')

In [178]:
# train = pd.read_csv('train.csv')
test = pd.read_csv('Test.csv')

In [179]:
test.head()

,Filename
0,Img1137.jpg
1,Img3633.jpg
2,Img1474.jpg
3,Img3459.jpg
4,Img5578.jpg


In [176]:
sample=np.random.choice(train.Images)
image=Image.open(os.path.join('/home/roshanbtech/Deep_Learning/Frenship_detection/'+'/train_combined/'+ sample))
plt.imshow(image)
plt.title(sample)

UnidentifiedImageError: cannot identify image file '/home/roshanbtech/Deep_Learning/Frenship_detection//train_combined/successful-group-happy-friends-on-260nw-408358303.jpg'

### Creating Training DataSet

In [187]:
def createFileList(myDir, format='.jpg'):
    fileList = []
    
    for root, dirs, files in os.walk(myDir, topdown=False):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
    return fileList

In [198]:
def create_dataframe(key):
    out = createFileList('/home/roshanbtech/Deep_Learning/Frenship_detection/data/train/'+ key)
    
    out = pd.DataFrame(out)
    split_data = out[0].str.split("/")
    data = split_data.to_list()
    data = [i[-1] for i in data]
    train = pd.DataFrame(data)
    train = train.rename(columns = {0:'Images'})
    train['label'] = key.lower()
    return train

In [199]:
train_adults = create_dataframe('Adults')
train_teenagers = create_dataframe('Teenagers')
train_toddlers = create_dataframe('Toddlers')

In [200]:
train_adults.sample(5)

,Images,label
352,0012.jpg,adults
400,00000013.jpg,adults
337,00000185.jpg,adults
114,00000436.jpg,adults
358,0014.jpg,adults


In [201]:
train_teenagers.sample(5)

,Images,label
483,00000312.jpg,teenagers
326,young-people-pointing-you-smiling-41531521.jpg,teenagers
417,00000304.jpg,teenagers
321,00000200.jpg,teenagers
72,00000181.jpg,teenagers


In [202]:
train_toddlers.sample(5)

,Images,label
448,00000526.jpg,toddlers
39,00000087.jpg,toddlers
491,00000424.jpg,toddlers
295,00000062.jpg,toddlers
389,children-natural-playground-near-smartinsko-la...,toddlers


In [205]:
# Concatinating three datasets
train = pd.concat([train_adults,train_teenagers,train_toddlers])

In [206]:
train.shape

(1429, 2)

In [207]:
#As the Images have repeated names for different labels, so we are removing duplicate images and retaining images with label adult
train.drop_duplicates(subset=['Images'], keep='first', inplace=True, ignore_index=False)

In [208]:
train.shape

(1149, 2)

In [209]:
train.sample(5)

,Images,label
391,118118306-portrait-of-asian-college-student-on...,teenagers
159,children-natural-playground-near-smartinsko-la...,toddlers
311,33474175-high-school-students-collaborating-on...,teenagers
236,group-diverse-kids-having-fun-together-park-gr...,toddlers
113,00000137.jpg,adults


In [213]:
from sklearn.model_selection import train_test_split
train_df,valid_df=train_test_split(train,test_size=0.3,shuffle=True)

In [214]:
datagen = ImageDataGenerator(rescale=1./255.,
                             rotation_range=20,
                             width_shift_range=0.2, 
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True,
                             fill_mode='nearest',
                            )


train_generator = datagen.flow_from_dataframe(
dataframe=train_df,
directory="/home/roshanbtech/Deep_Learning/Frenship_detection/data/train/train_combined",
x_col="Images",
y_col="label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 804 validated image filenames belonging to 3 classes.


In [215]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    valid_df, 
    directory="/home/roshanbtech/Deep_Learning/Frenship_detection/data/train/train_combined",
x_col="Images",
y_col="label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))


Found 345 validated image filenames belonging to 3 classes.


In [216]:
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
dataframe=test,
directory="/home/roshanbtech/Deep_Learning/Frenship_detection/Test Data",
x_col="Filename",
y_col=None,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(224,224))

Found 314 validated image filenames.


In [217]:
vgg_model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (224,224, 3))

In [218]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [219]:
model = Sequential()

model.add(vgg_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              25691136  
_________________________________________________________________
dropout_11 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 3075      
Total params: 40,408,899
Trainable params: 40,408,899
Non-trainable params: 0
_________________________________________________________________


In [220]:
from keras import losses, optimizers, metrics
vgg_model.trainable = False
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

model.compile(
    optimizer=optimizers.Adam(lr=5e-5), 
    loss='categorical_crossentropy', 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=300, 
                              epochs=5, 
                              validation_data=validation_generator, 
                              validation_steps=12)

This is the number of trainable weights before freezing the conv base: 4
Epoch 1/5
300/300 [==============================] - 632s 2s/step - loss: 0.9200 - categorical_accuracy: 0.5699 - val_loss: 0.8215 - val_categorical_accuracy: 0.6048
Epoch 2/5
300/300 [==============================] - 632s 2s/step - loss: 0.7570 - categorical_accuracy: 0.6590 - val_loss: 0.7795 - val_categorical_accuracy: 0.5517
Epoch 3/5
300/300 [==============================] - 633s 2s/step - loss: 0.7096 - categorical_accuracy: 0.6869 - val_loss: 0.8955 - val_categorical_accuracy: 0.5570
Epoch 4/5
300/300 [==============================] - 631s 2s/step - loss: 0.6504 - categorical_accuracy: 0.7123 - val_loss: 0.9296 - val_categorical_accuracy: 0.5756
Epoch 5/5
300/300 [==============================] - 633s 2s/step - loss: 0.5805 - categorical_accuracy: 0.7542 - val_loss: 0.7806 - val_categorical_accuracy: 0.6419


In [221]:
test_generator.reset()
pred = model.predict_generator(test_generator,
                               steps=STEP_SIZE_TEST,
                               verbose=1)

314/314 [==============================] - 25s 79ms/step


In [222]:
predicted_class_indices=np.argmax(pred, axis=1)

In [223]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [224]:
predictions

['adults',
 'adults',
 'teenagers',
 'teenagers',
 'toddlers',
 'teenagers',
 'teenagers',
 'toddlers',
 'toddlers',
 'adults',
 'adults',
 'toddlers',
 'toddlers',
 'adults',
 'adults',
 'teenagers',
 'teenagers',
 'toddlers',
 'teenagers',
 'toddlers',
 'teenagers',
 'adults',
 'adults',
 'teenagers',
 'adults',
 'adults',
 'teenagers',
 'adults',
 'adults',
 'teenagers',
 'adults',
 'teenagers',
 'teenagers',
 'teenagers',
 'teenagers',
 'adults',
 'toddlers',
 'toddlers',
 'teenagers',
 'adults',
 'teenagers',
 'adults',
 'adults',
 'teenagers',
 'toddlers',
 'adults',
 'adults',
 'teenagers',
 'teenagers',
 'teenagers',
 'teenagers',
 'teenagers',
 'toddlers',
 'adults',
 'adults',
 'toddlers',
 'adults',
 'adults',
 'toddlers',
 'teenagers',
 'teenagers',
 'adults',
 'toddlers',
 'teenagers',
 'teenagers',
 'adults',
 'teenagers',
 'teenagers',
 'teenagers',
 'teenagers',
 'toddlers',
 'adults',
 'adults',
 'teenagers',
 'toddlers',
 'adults',
 'teenagers',
 'teenagers',
 'teenag

In [225]:
submission = pd.read_csv('Sample Submission.csv')

In [228]:
submission.head()

,Filename,Category
0,Img1137.jpg,Teenagers
1,Img3633.jpg,Toddler
2,Img1474.jpg,Adults
3,Img3459.jpg,Teenagers
4,Img5578.jpg,Toddler


In [229]:
submission['Category'] = predictions

In [230]:
submission.to_csv('sub.csv',index= False)